In [1]:
import numpy as np
import tensorflow as tf
from core import *

In [2]:
IMG_DIM = 64
STRIP_WIDTH = 512
STRIP_HEIGHT = 512
SLICE = 1
SNAPS = 8 ##Number of snaps over 4 seconds
CHANNELS = 1
CLASSES = 2
# CHANNELS = 8
conv1_filter = 4
conv2_filter = 4
conv3_filter = 8
conv4_filter = 8
conv5_filter = 16
conv6_filter = 16

In [3]:
experiment = '8snap'
path = './datasets/' + experiment + '/'

_04847_img = np.load(path + '4847_' + experiment + '-image.npy')
_04799_img = np.load(path + '4799_' + experiment + '-image.npy')
_04820_img = np.load(path + '4820_' + experiment + '-image.npy')
_05675_img = np.load(path + '5675_' + experiment + '-image.npy')
_05680_img = np.load(path + '5680_' + experiment + '-image.npy')
_05710_img = np.load(path + '5710_' + experiment + '-image.npy')

_04847_lbl = np.load(path + '4847_' + experiment + '-label-onehot.npy')
_04799_lbl = np.load(path + '4799_' + experiment + '-label-onehot.npy')
_04820_lbl = np.load(path + '4820_' + experiment + '-label-onehot.npy')
_05675_lbl = np.load(path + '5675_' + experiment + '-label-onehot.npy')
_05680_lbl = np.load(path + '5680_' + experiment + '-label-onehot.npy')
_05710_lbl = np.load(path + '5710_' + experiment + '-label-onehot.npy')

In [4]:

train_img = np.vstack((_04847_img[5:,], _04799_img[5:,], _04820_img[5:,], _05675_img[5:,], _05680_img[5:,]))
train_lbl = np.vstack((_04847_lbl[5:,], _04799_lbl[5:,], _04820_lbl[5:,], _05675_lbl[5:,], _05680_lbl[5:,]))
val_img = np.vstack((_04847_img[:5,], _04799_img[:5,], _04820_img[:5,], _05675_img[:5,], _05680_img[:5,]))
val_lbl = np.vstack((_04847_lbl[:5,], _04799_lbl[:5,], _04820_lbl[:5,], _05675_lbl[:5,], _05680_lbl[:5,]))
#val_img = _05710_img
#val_lbl = _05710_lbl

In [5]:
print 'train_img', train_img.shape
print 'train_lbl', train_lbl.shape
print 'val_img', val_img.shape
print 'val_lbl', val_lbl.shape

train_img (375, 512, 512)
train_lbl (375, 2)
val_img (25, 512, 512)
val_lbl (25, 2)


In [6]:
x = tf.placeholder(tf.float32, shape=[None, STRIP_HEIGHT, STRIP_WIDTH], name='x')
x_image = tf.reshape(x, [-1, STRIP_HEIGHT, STRIP_WIDTH, CHANNELS], name='x_reshaped')
y_ = tf.placeholder(tf.float32, shape=[None, CLASSES], name='labels')
phase_train = tf.placeholder(tf.bool, name='phase_train')
keep_prob = tf.placeholder(tf.float32, name='keep_prob_fc1')


# 3 convolutional layers
conv1 = conv_layer(x_image, CHANNELS, conv1_filter, phase_train, 'conv1', 8, 8)
conv2 = conv_layer(conv1, conv1_filter, conv2_filter, phase_train, 'conv2', 5, 5)
conv3 = conv_layer(conv2, conv2_filter, conv3_filter, phase_train, 'conv3', 3, 3)

# maxpool
max_pool_1 = max_pool_layer(conv3, name='max_pool_1')

# 2 convolutional layers
conv4 = conv_layer(max_pool_1, conv3_filter, conv4_filter, phase_train, 'conv4', 3, 3)
conv5 = conv_layer(conv4, conv4_filter, conv5_filter, phase_train, 'conv5', 4, 4)

# maxpool
max_pool_2 = max_pool_layer(conv5, name='max_pool_2')

conv6 = conv_layer(max_pool_2, conv5_filter, conv6_filter, phase_train, 'conv6', 4, 4)
max_pool_3 = max_pool_layer(conv6, name='max_pool_3')

max_pool_3_flat = tf.reshape(max_pool_3, [-1, 64 * 64 * conv6_filter]) # flatten 

fc1 = fc_layer(max_pool_3_flat, 64 * 64 * conv6_filter, 1024, phase_train, 'fc1') # fc_layer fc1
fc1_drop = tf.nn.dropout(fc1, keep_prob) # dropout on fc1
fc2 = fc_layer(tf.nn.relu(fc1_drop), 1024, 512, phase_train, name='fc2') # fc_layer fc2
fc3 = fc_layer(tf.nn.relu(fc2), 512, 256, phase_train, name='fc3') # fc_layer fc3
fc4 = fc_layer(tf.nn.relu(fc3), 256, 128, phase_train, name='fc4') # fc_layer fc4
fc5 = fc_layer(tf.nn.relu(fc4), 128, 64, phase_train, name='fc5') # fc_layer fc5
y_conv = fc_layer(fc5, 64, 2, phase_train, 'y_conv')    

In [ ]:
max_epochs = 200
starter_learning_rate = 1e-5
step = 400
decay = 0.5

global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                step, decay, staircase=True)

step, train_acc_arr, train_loss_arr, val_acc_arr, val_loss_arr = 0, [], [], [], []

with tf.name_scope('xent'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    cross_entropy_count = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy,
                                                                global_step = global_step)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_count = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('cross_entropy', cross_entropy)
tf.summary.scalar('accuracy', accuracy)
# tf.summary.image('input', x_image, 3)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [ ]:
sess = tf.InteractiveSession()
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter('/tmp/brain/4')
writer.add_graph(sess.graph)

sess.run(tf.global_variables_initializer())

for i in range(max_epochs):
    for batch_xs, batch_ys in next_batch(train_img, train_lbl, 5):
        feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 1.0, phase_train: True}
#         if step % 5 == 0:
#             s = sess.run(merged_summary, feed_dict=feed_dict)
#             writer.add_summary(s, step)

        train_step.run(feed_dict=feed_dict)
        step += 1

        if step % 100 == 0:
            
            train_loss, train_accuracy = loss_accuracy(cross_entropy_count, accuracy_count, x, y_, keep_prob, phase_train, train_img, train_lbl, 5)
            val_loss, val_accuracy = loss_accuracy(cross_entropy_count, accuracy_count, x, y_, keep_prob, phase_train, val_img, val_lbl, 5)
            
            train_acc_arr.append(train_accuracy), train_loss_arr.append(train_loss)
            val_acc_arr.append(val_accuracy), val_loss_arr.append(val_loss)

            print learning_rate.eval()
            print "step %d, train accuracy %.4f, train loss %.5f, val accuracy %g, val loss %g" % (
                step, train_accuracy, train_loss, val_accuracy, val_loss)

Current learning rate: 0.00001
step 100, train accuracy 0.5360, train loss 0.76645, val accuracy 0.52, val loss 0.897393
Current learning rate: 0.00001
step 200, train accuracy 0.7707, train loss 0.56728, val accuracy 0.72, val loss 0.632643
Current learning rate: 0.00001
step 300, train accuracy 0.6133, train loss 0.61279, val accuracy 0.56, val loss 0.615041
Current learning rate: 0.00000
step 400, train accuracy 0.6027, train loss 0.69427, val accuracy 0.56, val loss 0.691548
Current learning rate: 0.00000
step 500, train accuracy 0.8773, train loss 0.42213, val accuracy 0.68, val loss 0.687557
Current learning rate: 0.00000
step 600, train accuracy 0.7893, train loss 0.44088, val accuracy 0.68, val loss 0.61111
Current learning rate: 0.00000
step 700, train accuracy 0.9253, train loss 0.33367, val accuracy 0.72, val loss 0.594096
Current learning rate: 0.00000
step 800, train accuracy 0.7760, train loss 0.40327, val accuracy 0.52, val loss 0.64126
Current learning rate: 0.00000
ste

In [ ]:
test_loss, test_accuracy = loss_accuracy(cross_entropy_count, accuracy_count, x, y_, keep_prob, phase_train,
                                         val_img, val_lbl, 100)
print 'Val Accuracy: %g, Val Loss: %g' % (test_accuracy * 100, test_loss)

In [ ]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
get_available_devices()

In [ ]:
val_img.shape

In [ ]:
val_img[1,:,:,1].shape

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
for i in range(8):
    plt.show(plt.imshow(val_img[1,:,:,i]))

In [ ]:
plt.show(plt.imshow(np.concatenate(val_img[1], axis=1)))

In [ ]:
plt.show(plt.imshow(np.concatenate((val_img[1,:,:,0], val_img[1,:,:,1], val_img[1,:,:,2], val_img[1,:,:,3], val_img[1,:,:,4], val_img[1,:,:,5], val_img[1,:,:,6], val_img[1,:,:,7]), axis=1)))

In [ ]:
plt.show(plt.imshow(np.concatenate(val_img[1,:,:,[0,1,2,3,4,5,6,7]], axis=1)))

In [ ]:
val_img[1].T.T.T.shape